In [1]:
# Imports:
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import matplotlib.cm as cm
import seaborn as sns
from sklearn.metrics import *
sns.set()
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

length = 277
import lime
import lime.lime_tabular
from tensorflow.keras.models import load_model


2024-03-21 16:01:39.383679: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 16:01:39.403344: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 16:01:39.403367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 16:01:39.403867: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-21 16:01:39.407367: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6466060840889766009
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5803868160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10671326534856521941
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-21 16:01:41.922286: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 16:01:41.943515: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 16:01:41.943547: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 16:01:42.183326: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 16:01:42.183362: I external/local_xla/xla/stream_executor

In [3]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values
    
print(train_values.shape)
print(test_values.shape)

# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]

X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_test = X_test.reshape(-1, X_train.shape[1], 1)

y_train = train_values[:,-2]
y_test = test_values[:,-2]

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

(38751, 277)
(72794, 277)


In [4]:
def focal_loss(gamma=2.0):
    def focal_loss_fixed(y_true, y_pred):
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        loss = -K.pow(1 - pt, gamma) * K.log(pt)
        return loss

    return focal_loss_fixed

In [6]:
model_files = ['./models/mlpmodel.h5', './models/cnnmodel.h5', './models/lstmmodel.h5']
model_names = ['MLP','CNN', 'LSTM']

for model_file, model_name in zip(model_files, model_names):
        if model_name == 'LSTM':
                model = load_model(model_file, custom_objects={'focal_loss_fixed': focal_loss})
        else:
                model = load_model(model_file)

        feature_importance_df = pd.DataFrame()

        for class_num in range(1, 7):

            N_test = test_values[test_values[:,-2]==class_num]

            Xs_test = N_test[:,:-2]
            ys_test = N_test[:len(N_test),-2]

            Xs_test1 = Xs_test.reshape(-1, X_test.shape[1], 1)
            ys_test1 = to_categorical(ys_test)


            ecg_normalized_all = []

            for n in range (0, len(Xs_test1)):
                ecg_normalized_temp = (Xs_test1[n, :, 0] - Xs_test1[n, :, 0].min()) / (Xs_test1[n, :, 0].max() - Xs_test1[n, :, 0].min())
                ecg_normalized_all.append(ecg_normalized_temp)
            ecg_normalized_all = np.array(ecg_normalized_all)

            ecg_class_mean = np.mean(ecg_normalized_all, axis=0)

            # Initialize empty lists to store the weights and features
            all_weights = []
            all_features = []

            # Repeat the code block 100 times
            for _ in range(100):
                # Your code block here
                explainer = lime.lime_tabular.RecurrentTabularExplainer(X_train, feature_names=['x'], discretize_continuous=True)
                exp = explainer.explain_instance(ecg_class_mean, model.predict, labels=[class_num], num_features=275)

                # Get the weights for all features
                map_explanation = exp.as_map()[class_num]
                features, weights = [], []
                for e in map_explanation:
                    features.append(e[0])
                    weights.append(e[1])
                features = np.array(features)
                weights = np.array(weights)

                # Append the weights and features to the lists
                all_weights.append(weights)
                all_features.append(features)

            # Calculate the average weights and features
            average_weights = np.mean(all_weights, axis=0)
            average_features = np.mean(all_features, axis=0)

            # Update the average_weights and average_features as the new feature and weights

            features = average_features
            weights = average_weights

            # Sort based on features:
            indices_sort = np.argsort(features)
            features = features[indices_sort]
            weights = weights[indices_sort]

            # Average over the actual variables:
            mean_weights = np.zeros((23, ))
            for i in range(0, 275, 12):
                mean_weights[i // 12] = np.mean(weights[i:i+12])

            # Sort based on weight:
            indices_sort = np.argsort(-1 * mean_weights)
            slices = np.arange(1, 24)

            # Visualize:
            feature_importance_normalized = (mean_weights - mean_weights.min()) / (mean_weights.max() - mean_weights.min())

            fig, ax = plt.subplots(1, 2, figsize = (15, 4))
            ax[0].bar(range(23), feature_importance_normalized[indices_sort])
            ax[0].set_title('LIME weights for Synthetic ECG for class ' + str(class_num) + ' with a '+ str(model_name) + ' model')
            ax[0].set_xticks(np.arange(23))
            ax[0].set_xticklabels(slices[indices_sort])
            ax[0].set_ylabel('LIME weights')
            ax[0].set_xlabel('Segment')

            ecg_normalized = (ecg_class_mean - ecg_class_mean.min()) / ecg_class_mean.max() - ecg_class_mean.min()

            ax[1].plot(np.arange(275), ecg_normalized, label = 'Synthetic ECG for class ' + str(class_num))

            ax[1].plot(np.repeat(feature_importance_normalized , 12), label = 'LIME weight')
            ax[1].set_title('LIME weights for Synthetic ECG for class ' + str(class_num) + ' with a '+ str(model_name) + ' model')
            ax[1].set_ylabel('ECG signal / LIME weights')
            ax[1].set_xlabel('Time')
            ax[1].legend()
            plt.savefig('./output_pics/LIME weights for Synthetic ECG for class ' + str(class_num) + ' with a '+ str(model_name) + ' model', dpi = 400)

        # Append the importance_dict to the feature_importance_df DataFrame
            feature_importance_df['class_' + str(class_num)] = feature_importance_normalized
    # Save the feature_importance_df DataFrame to a CSV file
        feature_importance_df.to_csv('./output_csv/syn_lime_'+ str(model_name) + '.csv', index=False)


157/157 [==============================] - 0s 2ms/step
